In [14]:
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False

### 乱数シード(共通)

In [16]:
# 乱数シード (共通)
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

### 前処理クラス

In [17]:
# 前処理クラス
class ImageTransform():

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),  # ランダムクロップ
                transforms.RandomHorizontalFlip(),                       # ランダムフリップ(水平)
                transforms.ToTensor(),                                   # Tensor変換
                transforms.Normalize(mean, std)                          # 標準化
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),                               # リサイズ
                transforms.CenterCrop(resize),                           # クロップ
                transforms.ToTensor(),                                   # Tensor変換
                transforms.Normalize(mean, std)                                   # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

### データセットクラス

In [18]:
# アリとハチの画像データセットを作成
class HymenopteraDataset(data.Dataset):

    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase


    def __len__(self):
        return len(self.file_list)


    def __getitem__(self, index):

        img_path = self.file_list[index]
        img = Image.open(img_path)

        img_transformed = self.transform(img, self.phase) # torch.Size([3, 224, 224])

        if self.phase == 'train':
            label = img_path[30:34]
        elif self.phase == 'val':
            label = img_path[28:32]

        if label == 'ants':
            label = 0
        elif label == 'bees':
            label = 1

        return img_transformed, label

### ファイルパスリスト作成

In [19]:
# アリとハチの画像へのファイルパスのリストを作成する
def make_datapath_list(phase='train'):

    rootpath = "./data/Pytorch/hymenoptera_data/"
    target_path = osp.join(rootpath + phase + '/**/*.jpg')
    print(target_path)

    # ファイルパスを取得
    path_list = []
    for path in glob.glob(target_path):
        path_list.append(path)

    for i, path in enumerate(path_list):
        print(str(i) + ": " +  path)

    return path_list

### データセット作成

In [20]:
# データセット作成
def makeDataset():

    # 訓練
    train_list = make_datapath_list(phase='train')

    # 検証
    val_list = make_datapath_list(phase='val')

    # データセット
    resize = 224
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    transform = ImageTransform(resize, mean, std)
    train_dataset = HymenopteraDataset(file_list=train_list, transform=transform, phase='train')
    val_dataset = HymenopteraDataset(file_list=val_list, transform=transform, phase='val')

    # 動作確認
    index = 0
    print(train_dataset.__getitem__(index)[0].size()) # img_transformed
    print(train_dataset.__getitem__(index)[1])        # label

    print("Training Data: ")
    for train_index in range(0, train_dataset.__len__()):
        print("train" + "_" + str(train_index) +  " :" + str(train_dataset.__getitem__(train_index)[1]))

    return train_dataset, val_dataset

### データローダー

In [21]:
def dataLoader(train_dataset = None, val_dataset = None):
    
    # ミニバッチサイズ
    batch_size  = 32

    # DataLoader作成
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size = batch_size, shuffle = True
    )

    val_dataloader = torch.utils.data.DataLoader(
        val_dataset, batch_size = batch_size, shuffle = True
    )

    # 辞書にまとめる
    datasetloaders_dict = { 'train': train_dataloader, 'val': val_dataloader }

    # 動作確認
    batch_iterator = iter(datasetloaders_dict['train']) # イテレータに変換
    inputs, labels = next(batch_iterator) # 1番目の要素を取り出す
    print(inputs.size())
    print(labels)

    return datasetloaders_dict

### トレーニングと検証

In [22]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('--------')

        # epochごとの学習と検証ループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() # 訓練モードのモデル
            else:
                net.eval()  # 検証モードのモデル

            epoch_loss = 0.0   # epochの損失和
            epoch_corrects = 0 # epochの正解数 

            # 未学習時の検証性能を確かめるため、epoch=0の訓練では未学習のモデルを使用する
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # optimzerを初期化
                optimizer.zero_grad()

                # forward計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)              # 出力
                    loss = criterion(outputs, labels)  # 誤差
                    _, preds = torch.max(outputs, 1)   # 予測ラベル

                    # 訓練時はbackpropagation
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # イテレーションの計算結果
                    # lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0) # 平均loss * バッチ数
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


# ベタにファインチューニングプログラムを書く

In [36]:
# データセット
train_dataset, val_dataset = makeDataset()

# データローダー
dataloaders_dict = dataLoader(train_dataset=train_dataset, val_dataset=val_dataset)

./data/Pytorch/hymenoptera_data/train/**/*.jpg
0: ./data/Pytorch/hymenoptera_data/train/bees/2638074627_6b3ae746a0.jpg
1: ./data/Pytorch/hymenoptera_data/train/bees/507288830_f46e8d4cb2.jpg
2: ./data/Pytorch/hymenoptera_data/train/bees/2405441001_b06c36fa72.jpg
3: ./data/Pytorch/hymenoptera_data/train/bees/2962405283_22718d9617.jpg
4: ./data/Pytorch/hymenoptera_data/train/bees/446296270_d9e8b93ecf.jpg
5: ./data/Pytorch/hymenoptera_data/train/bees/1092977343_cb42b38d62.jpg
6: ./data/Pytorch/hymenoptera_data/train/bees/2704348794_eb5d5178c2.jpg
7: ./data/Pytorch/hymenoptera_data/train/bees/2358061370_9daabbd9ac.jpg
8: ./data/Pytorch/hymenoptera_data/train/bees/2861002136_52c7c6f708.jpg
9: ./data/Pytorch/hymenoptera_data/train/bees/266644509_d30bb16a1b.jpg
10: ./data/Pytorch/hymenoptera_data/train/bees/2470492904_837e97800d.jpg
11: ./data/Pytorch/hymenoptera_data/train/bees/2053200300_8911ef438a.jpg
12: ./data/Pytorch/hymenoptera_data/train/bees/2601176055_8464e6aa71.jpg
13: ./data/Pytorc

train_21 :a/tr
train_22 :a/tr
train_23 :a/tr
train_24 :a/tr
train_25 :a/tr
train_26 :a/tr
train_27 :a/tr
train_28 :a/tr
train_29 :a/tr
train_30 :a/tr
train_31 :a/tr
train_32 :a/tr
train_33 :a/tr
train_34 :a/tr
train_35 :a/tr
train_36 :a/tr
train_37 :a/tr
train_38 :a/tr
train_39 :a/tr
train_40 :a/tr
train_41 :a/tr
train_42 :a/tr
train_43 :a/tr
train_44 :a/tr
train_45 :a/tr
train_46 :a/tr
train_47 :a/tr
train_48 :a/tr
train_49 :a/tr
train_50 :a/tr
train_51 :a/tr
train_52 :a/tr
train_53 :a/tr
train_54 :a/tr
train_55 :a/tr
train_56 :a/tr
train_57 :a/tr
train_58 :a/tr
train_59 :a/tr
train_60 :a/tr
train_61 :a/tr
train_62 :a/tr
train_63 :a/tr
train_64 :a/tr
train_65 :a/tr
train_66 :a/tr
train_67 :a/tr
train_68 :a/tr
train_69 :a/tr
train_70 :a/tr
train_71 :a/tr
train_72 :a/tr
train_73 :a/tr
train_74 :a/tr
train_75 :a/tr
train_76 :a/tr
train_77 :a/tr
train_78 :a/tr
train_79 :a/tr
train_80 :a/tr
train_81 :a/tr
train_82 :a/tr
train_83 :a/tr
train_84 :a/tr
train_85 :a/tr
train_86 :a/tr
train_87 :

### ネットワークモデル

In [37]:
# ネットワークモデル作成
use_pretained = True
net = models.vgg16(pretrained=use_pretained) # 学習済みモデルVGG-16をロード
net.classifier[6] = nn.Linear(in_features=4096, out_features=2) # 出力層の出力ユニットをアリとハチの2つに付け替える
net.train() # 訓練モードに設定
print('ネットワーク設定完了: 学習済み重みを読み込み、訓練モードに設定しました。')

 # 損失関数の設定
criterion = nn.CrossEntropyLoss()

ネットワーク設定完了: 学習済み重みを読み込み、訓練モードに設定しました。


### ファインチューニング

In [38]:
# 異なる学習率毎にlistを作成
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

In [39]:
# 学習させる層のパラメータ名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

print(update_param_names_1)
print(update_param_names_1[0])

['features']
features


In [40]:
# パラメータ毎に各リストに格納する
for name, param, in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納: ", name)
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納:", name)
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納:", name)
    else:
        param.requires_grad = False
        print("勾配計算なし。学習しない:", name)

params_to_update_1に格納:  features.0.weight
params_to_update_1に格納:  features.0.bias
params_to_update_1に格納:  features.2.weight
params_to_update_1に格納:  features.2.bias
params_to_update_1に格納:  features.5.weight
params_to_update_1に格納:  features.5.bias
params_to_update_1に格納:  features.7.weight
params_to_update_1に格納:  features.7.bias
params_to_update_1に格納:  features.10.weight
params_to_update_1に格納:  features.10.bias
params_to_update_1に格納:  features.12.weight
params_to_update_1に格納:  features.12.bias
params_to_update_1に格納:  features.14.weight
params_to_update_1に格納:  features.14.bias
params_to_update_1に格納:  features.17.weight
params_to_update_1に格納:  features.17.bias
params_to_update_1に格納:  features.19.weight
params_to_update_1に格納:  features.19.bias
params_to_update_1に格納:  features.21.weight
params_to_update_1に格納:  features.21.bias
params_to_update_1に格納:  features.24.weight
params_to_update_1に格納:  features.24.bias
params_to_update_1に格納:  features.26.weight
params_to_update_1に格納:  features.26.bias


In [41]:
# 最適化手法の設定
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

In [42]:
# 初期設定
# GPUが使えるか確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス: ", device)

# ネットワークをGPUへ転送
net.to(device)

# ネットワークがある程度固定であれば、高速化される
torch.backends.cudnn.benchmark = True

# エポック
num_epochs = 2

使用デバイス:  cpu


In [43]:
# epochのループ
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('--------')

    # epochごとの学習と検証ループ
    for phase in ['train', 'val']:
        if phase == 'train':
            net.train() # 訓練モードのモデル
        else:
            net.eval()  # 検証モードのモデル

        epoch_loss = 0.0   # epochの損失和
        epoch_corrects = 0 # epochの正解数 

        # 未学習時の検証性能を確かめるため、epoch=0の訓練では未学習のモデルを使用する
        if (epoch == 0) and (phase == 'train'):
            continue

        # データローダからミニバッチを取り出すループ
        for inputs, labels in tqdm(dataloaders_dict[phase]):

            # GPUが使えるならGPUにデータを転送
            inputs = inputs.to(device)
            labels = labels.to(device)

            # optimzerを初期化
            optimizer.zero_grad()

            # forward計算
            with torch.set_grad_enabled(phase == 'train'):
                outputs = net(inputs)              # 出力
                loss = criterion(outputs, labels)  # 誤差
                _, preds = torch.max(outputs, 1)   # 予測ラベル

                # 訓練時はbackpropagation
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # イテレーションの計算結果
                # lossの合計を更新
                epoch_loss += loss.item() * inputs.size(0) # 平均loss * バッチ数
                # 正解数の合計を更新
                epoch_corrects += torch.sum(preds == labels.data)

        # epochごとのlossと正解率を表示
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/2
--------


  0%|          | 0/5 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'to'

### モデルの保存

In [45]:
"""モデルの保存"""
save_path = './data/Pytorch/finetuning_weight/weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

### モデルの読み込み

In [48]:
"""モデルの読み込み"""
load_path = "./data/Pytorch/finetuning_weight/weights_fine_tuning.pth"
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

<All keys matched successfully>

### GPU -> CPU

In [49]:
# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)

<All keys matched successfully>